In [1]:
import os
import cv2
import numpy as np
import math
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf

In [2]:
train_folder = 'train'
files = os.listdir('train')
train_data = []

for file in files:
    file_path = 'train/' + file
    img =cv2.imread(file_path)
    img = np.array(img)
    img = cv2.resize(img, (224,224))
    train_data.append(img)
    
train_data = np.array(train_data)
train_data = np.asarray(train_data, dtype = np.float32)

In [3]:
label_path = 'train_label.txt'
train_labels = []
with open(label_path,'r') as file:
    for line in file:
        label = line[1:4]
        if label == 'dog':
            train_labels.append(0)
        else:
            train_labels.append(1)
train_labels = np.array(train_labels)
train_labels = np.asarray(train_labels, dtype=np.int32)

In [4]:
eval_data = train_data[:200,:,:,:]
eval_labels = train_labels[:200]
train_data = train_data[200:,:,:,:]
train_labels = train_labels[200:]
print(train_data.shape)

(1800, 224, 224, 3)


In [6]:
def cnn_model(features, labels, mode):
    #Input Layer
    
    input_layer = features["x"]
    #Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32,
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.relu)
    
    #Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(
        inputs = conv1,
        pool_size = [2, 2],
        strides = 2)
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
    inputs = pool1,
    filters = 64,
    kernel_size = [5, 5],
    padding = "same",
    activation = tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    #Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 56 * 56 * 64])
    dense = tf.layers.dense(inputs = pool2_flat, units = 1024, activation = tf.nn.relu)
    dropout = tf.layers.dropout(inputs = dense, rate = 0.2, training = mode==tf.estimator.ModeKeys.TRAIN)
    #Logits Layer
    logits = tf.layers.dense(inputs = dropout, units = 2)
    
    predictions = {
        "classes": tf.argmax(input = logits, axis = 1),
        "probabilities": tf.nn.softmax(logits, name = "softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels = labels, logits = logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0001)
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op=train_op)
    
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(
            labels = labels, predictions = predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [7]:
pets_classifier = tf.estimator.Estimator(model_fn = cnn_model, model_dir = "pets_convnet_model")
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors = tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'pets_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000242C47E9978>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=10,
    num_epochs=None,
    shuffle=True)
pets_classifier.train(input_fn = train_input_fn, steps = 1000, hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into pets_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
INFO:tensorflow:loss = 11.866267, step = 1
INFO:tensorflow:probabilities = [[0.90401447 0.09598556]
 [0.8908366  0.10916343]
 [0.20778534 0.79221475]
 [0.63900626 0.36099377]
 [0.9285506  0.07144938]
 [0.8774996  0.12250045]
 [0.8712956  0.12870444]
 [0.5037758  0.49622422]
 [0.6847141  0.31528592]
 [0.71872103 0.28127897]] (92.004 sec)
INFO:tensorflow:global_step/sec: 0.537037
INFO:tensorflow:probabilities = [[0.6122364  0.38776365]
 [0.44353274 0.55646724]
 [0.38271704 0.617283  ]
 [0.6573808  0.34261915]
 [0.40705836 0.5929416 ]
 [0.3823006  0.61769944]
 [0.5555731  0.44442692]
 [0.45099983 0.54900014]
 [0.6767062  0.3232939 ]
 [0.6680718  0.3319282 ]] (94.210 sec)
INFO:tensorflow:loss = 0.7203934, step = 101 (186.221 sec)
INFO:tensorflow:prob

In [9]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = pets_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-02-26-20:24:21
INFO:tensorflow:Restoring parameters from pets_convnet_model\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-02-26-20:24:33
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.59, global_step = 1000, loss = 0.8727225
{'accuracy': 0.59, 'loss': 0.8727225, 'global_step': 1000}


In [11]:
print(eval_labels.sum()/200)
print(train_labels.sum()/1800)


0.65
0.6766666666666666
